# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Pip Installs

In [ ]:
!pip install gradio
!pip install easyocr
!pip install moviepy --upgrade
!pip install opencv - python
!pip install numpy
!pip install tqdm

# Imports

In [ ]:
import gradio as gr
import easyocr
import cv2
from collections import Counter
import subprocess
import os
from moviepy import VideoFileClip
import os
import math
from tqdm import tqdm

# Process Video

In [ ]:
def process_single_video(video_path):
    # Sets up Easy OCR model
    screen_reader = easyocr.Reader(['en'], gpu=True) #specify the language(s) and sets up the reader

    # Sets up video
    video = cv2.VideoCapture(video_path)

    # Gets total frames from the video to have a progress bar in termainal
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total=total_frames, desc="Processing frames")

    # array to hold every word
    all_texts = []
    # array that only holds one instance of that word
    seen_words = []
    frame_count = 0
    # Loops through frames
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        # Process every 10th frame
        if frame_count % 10 == 0:
            # Text recognition on the frame
            result = screen_reader.readtext(frame)

            # Extract all text from the current frame
            current_frame_texts = [text[1] for text in result]

            # Adds current_frame_texts to
            all_texts.extend(current_frame_texts)

            for word in current_frame_texts:
                if word not in seen_words:
                    seen_words.append(word)

        frame_count += 1
        pbar.update(1)

    video.release()
    pbar.close()

    word_count = Counter(all_texts)

    # Filters the reoccurrening words of the most occurring words
    filtered_texts = [text for text in seen_words if word_count[text] >= 10] # adjust num as needed

    # Grabs all the words into one string
    summ_text = " ".join(filtered_texts)



    #convert transcript to segmented_transcript: an array of strings that divides transcript at topic changes
    paragraph_breaker_results = subprocess.run(['python', '/content/drive/MyDrive/VideoSummarizer/paragraph_breaker.py', summ_text, '--topical_summary'],
                                                capture_output=True,
                                                text=True,
                                                check=True)
    segmented_transcript = paragraph_breaker_results.stdout.strip()

    output_file = "visual_transcript.txt"
    with open(output_file, "w") as f:
        # Writes the summaries to a text file
        f.write(segmented_transcript)

    try:
        pegasus_test_result = subprocess.run(
            ['python', '/content/drive/MyDrive/VideoSummarizer/distilPegasus/pegasus_test.py', output_file],
            check=True,
            capture_output=True,
            text=True
        )
        summary = pegasus_test_result.stdout.strip()

    except subprocess.CalledProcessError as e:
        print("Pegasus summarization failed!")
        print("Exit code:", e.returncode)
        print("Error output:\n", e.stderr)
        summary = "Pegasus summarization failed"


    return summary

In [ ]:
def process_video(video_path):
    # Splits the video into segments if needed
    segments = split_video_into_segments(video_path)
    if not segments:
        segments = [video_path]

    # array for all visual summaries
    all_visual_summaries = []
    for i, segment in segments:
       print(f"Processing visual segment:{segment}")
       # Processes video segment to get the summary
       summary = process_single_video(segment)
       segment_label = f"--- Segment {i + 1}: {os.path.basename(segment)} ---"
       all_visual_summaries.append(f"{segment_label}\n{summary}")

    # Combines all the summaries into one string
    combined_summary = "\n\n".join(all_visual_summaries)

    output_file = "visual_summary.txt"
    with open(output_file, "w") as f:
        # Writes the summaries to a text file
        f.write(combined_summary)

    return combined_summary, output_file

# Process Audio

In [ ]:
def process_single_audio(audio_path):
    #run whisper_test.py, get printed transcript from audio
    whisper_test_result = subprocess.run(['python', '/content/drive/MyDrive/VideoSummarizer/distilWhisper/whisper_test.py', audio_path],
                                            capture_output=True,
                                            text=True,
                                            check=True)
    transcript = whisper_test_result.stdout.strip()
    #print(f"Audio Transcript:\n{transcript}")

    #print(f"Transcript:\n{transcript}\n")

    #convert transcript to segmented_transcript: an array of strings that divides transcript at topic changes
    paragraph_breaker_results = subprocess.run(['python', '/content/drive/MyDrive/VideoSummarizer/paragraph_breaker.py', transcript, '--topical_summary'],
                                                capture_output=True,
                                                text=True,
                                                check=True)
    segmented_transcript = paragraph_breaker_results.stdout.strip()

    output_file = "audio_transcript.txt"
    with open(output_file, "w") as f:
        # Writes the summaries to a text file
        f.write(segmented_transcript)

    #pass segmented_transcript to pegasus_test.py and get summary
    pegasus_test_result = subprocess.run(['python', '/content/drive/MyDrive/VideoSummarizer/distilPegasus/pegasus_test.py', output_file],
                                            check=True,
                                            capture_output=True,
                                            text=True)
    summary = pegasus_test_result.stdout.strip()

    return summary


In [ ]:
def process_audio(video_path):
    # Splits the video into segments if needed
    segments = split_video_into_segments(video_path)
    if not segments:
        segments = [video_path]

    # array for all audio summaries
    all_audio_summaries = []
    for i, segment in enumerate(segments):
        print(f"Processing audio segment: {segment}")
        # Processes video segment to get the summary
        summary = process_single_audio(segment)
        segment_label = f"--- Segment {i + 1}: {os.path.basename(segment)} ---"
        all_audio_summaries.append(f"{segment_label}\n{summary}")

    # Combines all the summaries into one string
    combined_summary = "\n\n".join(all_audio_summaries)

    output_file = "audio_summary.txt"
    with open(output_file,"w") as f:
        # Writes the summaries to a text file
        f.write(combined_summary)

    return combined_summary, output_file


# Splice Video

In [ ]:
def split_video_into_segments(input_path, segment_minutes=5):
    video = VideoFileClip(input_path)
    duration = video.duration  # in seconds

    # Checks if the duration of the video is less than 15 minutes
    if duration <= segment_minutes * 60:
        print("Video is shorter than or equal to 15 minutes. No split needed.")
        return []

    # Gets the total number of segements of the video
    total_segments = math.ceil(duration / (segment_minutes * 60))
    base_filename = os.path.splitext(os.path.basename(input_path))[0]

    output_files = []

    for i in range(total_segments):
        # Breaks the video into segments
        start_time = i * segment_minutes * 60
        end_time = min((i + 1) * segment_minutes * 60, duration)
        segment = video.subclipped(start_time, end_time)

        # Saves Video Segments
        output_path = f"{base_filename}_part{i + 1}.mp4"
        segment.write_videofile(output_path, codec='libx264', audio_codec='aac', logger=None)

        output_files.append(output_path)
        print(f"Saved segment {i + 1}: {start_time:.2f}s to {end_time:.2f}s -> {output_path}")

    video.close()
    print("Splitting complete.")
    return output_files

# Process Video Segment

In [ ]:
def process_segments(input_path):
    segments = split_video_into_segments(input_path)

    if not segments:
        segments = [input_path]

    # Arrays for visual and audio summaries
    all_visual_summaries = []
    all_audio_summaries = []

    for i, segment in enumerate(segments):
        print(f"Processing segment: {segment}")
        # Processes both audio and visual to get their summaries
        visual_summary = process_single_video(segment)
        audio_summary = process_single_audio(segment)

        segment_label = f"--- Segment {i + 1}: {os.path.basename(segment)} ---"
        all_visual_summaries.append(f"{segment_label}\n{visual_summary}")
        all_audio_summaries.append(f"{segment_label}\n{audio_summary}")

    # Combine summaries
    full_visual_summary = "\n\n".join(all_visual_summaries)
    full_audio_summary = "\n\n".join(all_audio_summaries)

    # Write combined summaries to new files
    combined_vis_file = "combined_visual_summary.txt"
    with open(combined_vis_file, "w") as f:
        f.write(full_visual_summary)

    combined_audio_file = "combined_audio_summary.txt"
    with open(combined_audio_file, "w") as f:
        f.write(full_audio_summary)

    # Safe guard incase one of the summaries are empty
    if not all_visual_summaries:
      full_visual_summary = "No visual summary generated."
      combined_vis_file = ""

    if not all_audio_summaries:
        full_audio_summary = "No audio summary generated."
        combined_audio_file = ""

    return full_visual_summary, combined_vis_file, full_audio_summary, combined_audio_file


# Gradio UI

In [ ]:
with gr.Blocks() as demo:
    with gr.Column():
      with gr.Row(equal_height=True):
        # Visual summary textbox, Upload a Video file, and Audio summary textbox
        left_textbox = gr.Textbox(label="Visual Summary", max_lines=20, scale=1)
        video_input = gr.File(label="Upload Video", scale=1)
        right_textbox = gr.Textbox(label="Audio Summary", max_lines=20, scale=1)


    with gr.Row():
        # Process buttons
        process_video_button = gr.Button("Process Video")
        process_full_button = gr.Button("Process Full Video (Audio + Visual)")
        process_audio_button = gr.Button("Process Audio")

    with gr.Row():
      # Download File
      download_vis_file = gr.File(label="Download Visual Summary", visible= False)
      download_audio_file = gr.File(label="Download Audio Summary", visible= False)

    # Click event for just visual
    process_video_button.click(
        process_video,
        inputs=video_input,
        outputs=[left_textbox, download_vis_file]
    ).then(
      lambda: gr.update(visible=True),
      outputs=download_vis_file
    )

    # Click event for just audio
    process_audio_button.click(
        process_audio,
        inputs=video_input,
        outputs=[right_textbox, download_audio_file]
    ).then(
      lambda: gr.update(visible=True),
      outputs=download_audio_file
    )

    # Click event for both audio and visual
    process_full_button.click(
        process_segments,
        inputs=video_input,
        outputs=[left_textbox, download_vis_file, right_textbox, download_audio_file]
    ).then(
      lambda: (gr.update(visible=True), gr.update(visible=True)),
      outputs=[download_vis_file, download_audio_file]
    )

demo.launch(debug=True)

# Sources




In [ ]:
#OCR in the video_model_ocr was based on Huma Gonen's Kraggle Notebook
#link: https://www.kaggle.com/code/humagonen/ocr-optical-character-recognition/notebook